In [ ]:
import torchlensmaker as tlm
import torch
import torch.nn as nn
from pprint import pprint

from torchlensmaker.testing.basic_transform import basic_transform
from torchlensmaker.core.transforms import IdentityTransform
from torchlensmaker.testing.collision_datasets import *
from torchlensmaker.core.collision_detection import Newton, GD, LM

import matplotlib.pyplot as plt

# TODO test/view F and F grad of samples2D
# F should be zero
# F grad should be finite

# new way to sample expected collide:
# start from samples2D, pick random V, offset by random amount

# analysis:
# given dataset expected collide
# list of algorithm
# for each algorithm:
# number of missing collisions
# distribution of number of iterations to converge

def dataset_view(dataset, rays_length=100):
    "View a collision dataset testcase with tlmviewer"

    # TODO display points at P to see rays origins

    scene = tlm.viewer.new_scene("2D")
    #scene["data"].extend(tlm.viewer.render_collisions(all_points, all_normals))

    rays_start = dataset.P - rays_length*dataset.V
    rays_end = dataset.P + rays_length*dataset.V
    scene["data"].append(
        tlm.viewer.render_rays(rays_start, rays_end, layer=0)
    )

    assert torch.all(torch.isfinite(dataset.P))
    assert torch.all(torch.isfinite(dataset.V))

    scene["data"].append(tlm.viewer.render_surfaces([dataset.surface], [IdentityTransform(dim=2, dtype=dataset.surface.dtype)], dim=2))
    scene["title"] = dataset.name
    tlm.viewer.ipython_display(scene)
    #tlm.viewer.dump(scene, ndigits=2)



def convergence_plot(dataset, algos):
    "Plot convergence of collision detection for multiple algorithms"

    surface = dataset.surface
    P, V = dataset.P, dataset.V

    # move rays by a tiny bit, to avoid t=0 local minimum
    # that happens with constructed datasets
    # TODO augment dataset with different shifts
    #P, V = move_rays(P, V, 0.)
    
    fig, axes = plt.subplots(len(algos), 1, figsize=(10, 3*len(algos)), layout="tight", squeeze=False)

    for i, algorithm in enumerate(algos):
        axQ = axes.flat[i]

        t_solve, t_history = algorithm(surface, P, V, init_t=torch.zeros((P.shape[0],)), history=True)
        
        # Reshape tensors for broadcasting
        N, H = P.shape[0], t_history.shape[1]
        P_expanded = P.unsqueeze(1)  # Shape: (N, 1, 2)
        V_expanded = V.unsqueeze(1)  # Shape: (N, 1, 2)
        t_history_expanded = t_history.unsqueeze(2)  # Shape: (N, H, 1)
    
        # Compute points_history
        points_history = P_expanded + t_history_expanded * V_expanded  # Shape: (N, H, 2)
    
        assert t_history.shape == (N, H), (N, H)
        assert points_history.shape == (N, H, 2)
    
        # plot Q(t)
        for ray_index in range(t_history.shape[0]):
            axQ.plot(range(t_history.shape[1]), surface.f(points_history[ray_index, :, :]))
        
        axQ.set_xlabel("iteration")
        axQ.set_ylabel("Q(t)", rotation=0)
        axQ.set_title(f"{dataset.name} | {str(algorithm)}")

    return fig


def collision_statistics(surface, dataset, algos):
    "Compute and return collision statistics for a dataset and an algorithm"

    P, V = dataset.P, dataset.V

    for i, algorithm in enumerate(algos):
        t_solve, t_history = algorithm(surface, P, V, init_t=torch.zeros((P.shape[0],)), history=True)
            
        # Reshape tensors for broadcasting
        N, H = P.shape[0], t_history.shape[1]
        P_expanded = P.unsqueeze(1)  # Shape: (N, 1, 2)
        V_expanded = V.unsqueeze(1)  # Shape: (N, 1, 2)
        t_history_expanded = t_history.unsqueeze(2)  # Shape: (N, H, 1)
    
        # Compute points_history
        points_history = P_expanded + t_history_expanded * V_expanded  # Shape: (N, H, 2)
    
        assert t_history.shape == (N, H), (N, H)
        assert points_history.shape == (N, H, 2)

        # count number of collisions
        local_points = points_history[:, -1, :]
        residuals = surface.f(local_points)

        tol = 1e-6
        count = torch.sum(torch.abs(residuals) > tol).item()
        error = torch.sqrt(torch.sum(residuals**2) / N).item()

        print(f"{str(algorithm): <20} error={error:.6f} ({count} misses)")

    return

algorithms = [
    Newton(0.8, max_iter=10, max_delta=10),
    GD(0.1, max_iter=10, max_delta=10),
    LM(1.0, max_iter=10, max_delta=10),
]

surface = Sphere(10, 10)

generators = [
    normal_rays(offset=3.0, N=25),
    offset_rays(offset=-0.2, N=15),
]

for gen in generators:
    dataset = gen(surface)

    # tlmviewer view
    dataset_view(dataset)

    # statistics
    collision_statistics(surface, dataset, algorithms)
    
    # convergence plots
    fig = convergence_plot(dataset, algorithms)
    plt.show(fig)

# individual test:
# 1 surface
# 1 ray generator

# batch test:
# 1 surface
# many ray generators

# merge of all datasets
#merged = merge_datasets(list(COLLISION_DATASET_REGISTRY.values()))
#dataset_view(merged)

In [ ]:
from torchlensmaker.core.surfaces import Sphere

from torch.nn.functional import normalize

s = Sphere(10, 5)
samples = s.samples2D_full(N=3, epsilon=0.)
grad = normalize(s.f_grad(samples))

print(samples)
print(grad)

samples[:, 1] <= 5

In [ ]:
torch.rand((5, 2)).shape == (5, 2)